In [1]:
from statsmodels.tsa.statespace.varmax import VARMAX
from random import random
import warnings
import pandas as pd
from pandas import read_csv, DataFrame
from datetime import datetime
from matplotlib import pyplot
from pandas.plotting import autocorrelation_plot
from sklearn.metrics import mean_squared_error
from math import sqrt

In [ ]:
series_train = read_csv('./Data/Processed_BT_PriceDB_train.csv', header=0, parse_dates=[0], index_col=0, squeeze=True)
series_test = read_csv('./Data/Processed_BT_PriceDB_test.csv', header=0, parse_dates=[0], index_col=0, squeeze=True)

In [ ]:
print(series_train.shape)
print(series_test.shape)

In [2]:
btc_price_df = pd.read_csv("./Data/BTC_price_db.csv")
tweets_df = pd.read_csv("./Data/tweets_db.csv")
merge_df = pd.merge(btc_price_df,tweets_df, on='time_')
merge_df = merge_df.drop(["time_"], axis=1)
merge_df.shape

(121, 4)

In [3]:
size = int(len(merge_df)*0.7)

# Divide into train and test
train, test = merge_df[0:size], merge_df[size:len(merge_df)]

print(len(train))
print(len(test))

84
37


In [ ]:
def evaluate_varmax_model(train, test, varmax_order):
    history = [x for x in train.values]
    test = [x for x in test.values]
    predictions = list()
    acutal = list()
    print(varmax_order)
    for t in range(len(test)):
        model = VARMAX(history, order=varmax_order)
        model_fit = model.fit()
        # make prediction
        yhat = model_fit.forecast()
        predictions.append(yhat[0][0])
        history.append(test[t])
        acutal.append(test[0][0])
    rmse = sqrt(mean_squared_error(acutal, predictions))
    return rmse


def evaluate_models(train, test, p_values, q_values):
    best_score, best_cfg = float("inf"), None
    for p in p_values:
        for q in q_values:
            order = (p,q)
            try:
                mse = evaluate_varmax_model(train, test, order)
                if mse < best_score:
                    best_score, best_cfg = mse, order
                    print('VARMAX%s RMSE=%.3f' % (order,mse))
            except:
                continue
    print('Best VARMAX%s RMSE=%.3f' % (best_cfg, best_score))

In [ ]:
p_values = [0, 1, 2, 4, 6, 8, 10]
q_values = range(0, 4)
warnings.filterwarnings("ignore")
evaluate_models(train, test, p_values, q_values)

In [ ]:
# contrived dataset with dependency
data = list()
for i in range(100):
    v1 = random()
    v2 = v1 + random()
    row = [v1, v2]
    data.append(row)

In [4]:
# fit model
history = [list(x) for x in train.values]
model = VARMAX(history, order=(1, 1))
model_fit = model.fit(disp=False)
# make prediction
yhat = model_fit.forecast()
print(yhat[0][0])

/Users/mz195/Library/Python/3.8/lib/python/site-packages/statsmodels/tsa/statespace/varmax.py:161: EstimationWarning: Estimation of VARMA(p,q) models is not generically robust, due especially to identification issues.
  warn('Estimation of VARMA(p,q) models is not generically robust,'


33423.606471755855
